## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''

'''Resnet 50 架構'''
base_model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
base_model.summary()


[0]              
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 4, 4, 512)    0           add_7[0][0]                      
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, 2, 2, 256)    131328      activation_22[0][0]              
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, 2, 2, 256)    1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 2, 2, 256)    0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_branch2b (Conv2D)         (None, 2, 2, 256)    590080      activation_23[0][0]       

## 添加層數

In [2]:
x = base_model.output

'''可以參考Cifar10實作章節'''
#predictions = GlobalAveragePooling2D()(x)
predictions = Flatten()(x)
predictions = Dense(1024, activation='relu')(predictions)
predictions = Dense(output_dim=10,activation='softmax')(predictions)
model = Model(inputs=base_model.input, outputs=predictions)
model_len=len(base_model.layers)
print(model_len)
print('Model深度：', len(model.layers))

175
Model深度： 178


In [3]:
model.summary()

][0]             
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 2, 2, 256)    0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_branch2b (Conv2D)         (None, 2, 2, 256)    590080      activation_23[0][0]              
__________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizati (None, 2, 2, 256)    1024        res4a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_24 (Activation)      (None, 2, 2, 256)    0           bn4a_branch2b[0][0]              
__________________________________________________________________________________________________
res4a_branch2c (Conv2D)         (None, 2, 2, 1024)   263168      activation_24[0][0]       

## 鎖定特定幾層不要更新權重

In [4]:
offset=5
for layer in model.layers[:model_len-offset]:
    layer.trainable = False
for layer in model.layers[model_len-offset:]:
    layer.trainable = True
for layer in model.layers:
    print(layer,layer.trainable)    

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = keras.optimizers.Adam(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=64,epochs=50)

Epoch 1/50
50000/50000 [==============================] - 34s 677us/step - loss: 1.7094 - accuracy: 0.4276
Epoch 2/50
50000/50000 [==============================] - 29s 577us/step - loss: 1.3483 - accuracy: 0.5367
Epoch 3/50
50000/50000 [==============================] - 29s 585us/step - loss: 1.2189 - accuracy: 0.5775
Epoch 4/50
50000/50000 [==============================] - 30s 610us/step - loss: 1.1276 - accuracy: 0.6086
Epoch 5/50
50000/50000 [==============================] - 30s 609us/step - loss: 1.0527 - accuracy: 0.6331
Epoch 6/50
50000/50000 [==============================] - 30s 609us/step - loss: 0.9799 - accuracy: 0.6624
Epoch 7/50
50000/50000 [==============================] - 30s 609us/step - loss: 0.9210 - accuracy: 0.6822
Epoch 8/50
50000/50000 [==============================] - 30s 609us/step - loss: 0.8690 - accuracy: 0.7028
Epoch 9/50
50000/50000 [==============================] - 30s 610us/step - loss: 0.8181 - accuracy: 0.7197
Epoch 10/50
50000/50000 [============